In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import markdownify
import re
from tqdm import tqdm

In [58]:
# url = "https://essayforum.com/writing/average-cell-phone-national-fixed-87878/"
# url = "https://essayforum.com/writing/people-think-students-study-science-food-94982/"
base_url = (
    "https://essayforum.com/writing/shortage-housing-big-cities-cause-severe-94815/"
)
response = requests.get(url=base_url)
soup = BeautifulSoup(response.text)
all_messages = soup.find_all(
    "article",
)

In [ ]:
import pprint

pprint.pprint(str(all_messages[0]))

In [59]:
author = all_messages[0].find("div", "fl").find_all("a")
if len(author) > 0:
    author = author[0].find("b").text
else:
    author = all_messages[0].find("div", "fl").text.replace("/", "").strip()
author

'Hazard'

In [37]:
number = all_messages[0].find("div", "fr").find("a").text
date = all_messages[0].find("div", "fr").text.replace(number, "").strip()
date

'3 hrs ago'

In [ ]:
message = str(all_messages[0].find("div", "pTx"))

message = markdownify.markdownify(message, heading_style="ATX")
message = re.sub("\!\[\]\(\w+.*\)", "", message).strip()
print(message)

In [41]:
for message in all_messages:
    author = message.find("div", "fl").find("a").find("b").text.strip()
    print(author)

    number = all_messages[0].find("div", "fr").find("a").text
    date = all_messages[0].find("div", "fr").text.replace(number, "").strip()
    print(date)

    message = str(message.find("div", "pTx"))
    message = markdownify.markdownify(message, heading_style="ATX")
    message = re.sub("\!\[\]\(\w+.*\)", "", message).strip()
    print(message)
    print("=" * 100)
    print("=" * 100)

huong1234
Jul 27, 2023
## Many women are choosing to have children later in life.

  
***Why is this? Do the advantages outweigh the disadvantages?***  
  
In modern society, there has been a major concern about the increasing number of women choosing to delay pregnancy. This essay attempts to depict plausible causes and shed light on both the merits and demerits of this phenomenon.  
  
In the past, most wives relied on the breadwinner-their husband, and stayed at home to do chores, give birth, and raise their children. Throughout industrial evolution, a number of changes have been made in the role of women in society, in which they have the right to make decisions themselves. Especially in democratic regions, this equality between genders has been considerably enhanced, which decisively raises the awareness of women about their own rights, leading to widespread non-child tendency in this sex. Moreover, due to globalization, the potential opportunities in occupations and education hav

In [62]:
class EssayForumMessagesParser:
    def __init__(self, urls) -> None:
        self.urls = [
            "https://essayforum.com/writing/increasing-number-women-choosing-delay-94853/",
            "https://essayforum.com/writing/shortage-housing-big-cities-cause-severe-94815/",
        ]

        self.dataset = []

    def parse(self):
        for url in self.urls:
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)
            all_messages = soup.find_all("article")
            dataset_item = []
            for pos, message in enumerate(all_messages):
                author = message.find("div", "fl").find_all("a")
                if len(author) > 0:
                    author = author[0].find("b").text
                else:
                    author = message.find("div", "fl").text.replace("/", "").strip()
                # print(author)

                number = all_messages[0].find("div", "fr").find("a").text
                date = (
                    all_messages[0].find("div", "fr").text.replace(number, "").strip()
                )
                # print(date)

                message = str(message.find("div", "pTx"))
                message = markdownify.markdownify(message, heading_style="ATX")
                message = re.sub("\!\[\]\(\w+.*\)", "", message).strip()
                # print(message)
                # print("=" * 100)
                # print("=" * 100)
                dataset_item.append(
                    {
                        "message": message,
                        "author": author,
                        "date": date,
                        "position": pos,
                        "url": url,
                    }
                )

            self.dataset.append(dataset_item)


parser = EssayForumMessagesParser(urls=[])
parser.parse()

In [63]:
parser.dataset

[[{'message': "## Many women are choosing to have children later in life.\n\n  \n***Why is this? Do the advantages outweigh the disadvantages?***  \n  \nIn modern society, there has been a major concern about the increasing number of women choosing to delay pregnancy. This essay attempts to depict plausible causes and shed light on both the merits and demerits of this phenomenon.  \n  \nIn the past, most wives relied on the breadwinner-their husband, and stayed at home to do chores, give birth, and raise their children. Throughout industrial evolution, a number of changes have been made in the role of women in society, in which they have the right to make decisions themselves. Especially in democratic regions, this equality between genders has been considerably enhanced, which decisively raises the awareness of women about their own rights, leading to widespread non-child tendency in this sex. Moreover, due to globalization, the potential opportunities in occupations and education have

### parse links


In [64]:
base_url = "https://essayforum.com/writing/2/"
response = requests.get(url=base_url)
soup = BeautifulSoup(response.text)
all_links = soup.find_all("a", class_="txtNr")

In [70]:
all_links[0]["href"]

'https://essayforum.com/writing/discuss-view-give-opinion-zoo-closed-due-94830/'

In [68]:
total_pages = soup.find_all("a", class_="navCell")[-1].text
total_pages = int(total_pages)

'478'

In [73]:
class EssayForumLinksParser:
    def __init__(self, urls) -> None:
        self.base_urls = urls

        self.dataset = []

    def parse(self):
        for base_url in self.base_urls:
            response = requests.get(url=base_url)
            soup = BeautifulSoup(response.text)

            total_pages = soup.find_all("a", class_="navCell")[-1].text
            total_pages = int(total_pages)

            forum_type = base_url.replace("https://", "")
            forum_type = forum_type.replace("/", "_")

            for pos in tqdm(range(1, total_pages + 1)):
                url = f"{base_url}/{pos}"
                response = requests.get(url=url)
                soup = BeautifulSoup(response.text)
                all_links = soup.find_all("a", class_="txtNr")

                for link in all_links:
                    self.dataset.append({"forum_type": forum_type, "url": link["href"]})


links_parser = EssayForumLinksParser(urls=["https://essayforum.com/writing"])
links_parser.parse()

100%|██████████| 478/478 [05:56<00:00,  1.34it/s]


In [75]:
len(links_parser.dataset)

28669

In [77]:
pd.DataFrame(links_parser.dataset).to_csv(
    "./verbalist/datasets/essayforum/essayforum_writing_links.csv",
    index=False,
)